I have discussed my work with my friends Vamsi and Nikhlesh 

***Data Science Homework-1***
--Sriman (22110124)

I have discussed my work with my friends Vamsi and Nikhlesh 

**Question-1**

For the k-means problem, we are required to show that there is at most a factor of four ratio between the optimal value when
we either require all cluster centers to be data points or allow arbitrary points to be centers.

So now as we know the objective value of the kmeans is summation of square of distance of points to their respective cluster over all clusters.

So we will keep our discussion to one cluster, and prove the required for one cluster, and then similar arguement will be valid for other clusters as well, proving the complete result

Now in a cluster C1, say O1 is a cluster center which is not a datapoint, and O2 is a cluster center which is a datapoint

Now couple of arguements can be made from the above setting

a. First we can say that O1 is the actual mean of all the data points present in the cluster, and O2 is the nearest data point to O1.

b. now for any point 'p' in the cluster C1, we can say that **d(O1,O2)<=d(O1,p)** (say equation-1).

c. now as we know the distance function 'd' used in k means is L2 norm, so we know it follows the triangle inequality, then we have 
    
**d(O1,O2)+d(O1,p)>=d(O2,p)** (say equation-2).

Now using equation-1 in equation-2 we will be having the result as follows

**d(O1,p)+d(O1,p)>=d(O2,p)**

**2d(O1,p)>=d(O2,p)** (say equation-3)

Now just sqauring the equation-3 and summing it over all clusters(as our discussion was on single cluster), we get our required equation as 

**4(OPT1)>=OPT2**

This above final equation proves our required result


**Question-2**

Now first let's write the markov's inequality

for a>0, **P[X>=a] <= E[X]/a**

So now our goal is to find a Random Variable, that satisfies a tight bound with markov's inequality

Let X be a random variable given by 

P(X)={
    
    1/2 for X = 1

    1/2 for X=0
}

E[X]=1/2 (by direct calculation)

so we will have the markov's inequality as **P[X>=a] <= (0.5)/a**

now let a=1, we will have 

**P[X>=1] <= 0.5**

But as we know that X can take only 0 or 1, so we can write **P[X=1] <=0.5**

so we know that P[X=1]=0.5, so we can see that the markov's inequality in this example give's the tightest bound


Now coming to second part of the question, why we use other inequalities?, because they provide tighter bounds for certain problems and have advantages than markov, so we need to choose the inequalities based on the problem 

We use chebyshev inequality, because it provides bounds on both the sides and this inequality provides a tighter bound when dealing with random variables with finite variance and in the case of chernoff inequality, it can be used to estimate bounds on probability of sum of independent variables



**Question-3**

In [1]:
pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.15.0 from https://files.pythonhosted.org/packages/4c/48/1a5a15517f18eaa4ff8d598b1c000300b20c1bb0e624539d702117a0c369/tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ----------------------------------- ---- 51.2


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()



In [43]:
test_images.shape##(10000, 28, 28)
test_images=test_images/255 ## normalizing the values 
##now reshaping the data to 2D vector
n_samples, height, width = test_images.shape
test_images = test_images.reshape(n_samples, height * width)
test_images.shape##(10000, 784)

(10000, 784)

**part a) Kmeans with Kmeans++ initialization**

In [14]:
from tqdm import tqdm
import numpy as np

def kmeans(x, k, max_iter):
    n, features = x.shape
    centroids = kmeans_pp_init(x, k) # initializing through kmeans++
    for iteration in tqdm(range(max_iter), desc="K-Means Progress"):
        distances = np.zeros((n, k))
        for i in range(n):
            for j in range(k):
                distances[i][j] = np.linalg.norm(x[i, :] - centroids[j])
        labels = np.argmin(distances, axis=1) # assigninig labels to all points to the nearest centroids
        for j in range(k):
            centroids[j] = np.mean(x[labels == j], axis=0) #updating the centroids to the mean of their respective clusters

    return centroids, labels

def kmeans_pp_init(x, k):
    centroids = [x[np.random.choice(len(x))]]
    for _ in range(1, k):
        distances = np.array([min([np.linalg.norm(c - point) for c in centroids]) for point in x])
        prob = distances / distances.sum()
        cum_prob = prob.cumsum()
        r = np.random.rand()
        for j, val in enumerate(cum_prob):
            if r < val:
                centroids.append(x[j])
                break
    return np.array(centroids)


from sklearn.metrics import rand_score

k = 10
max_iter = 20
centroids, labels = kmeans(test_images, k, max_iter)

rand_index = rand_score(test_labels, labels)
print("Rand Index:", rand_index)


K-Means Progress: 100%|██████████| 20/20 [00:08<00:00,  2.39it/s]

Rand Index: 0.9009117111711171


In [17]:
test_labels.shape

(10000,)

**part b) kcenter using greedy approach**

In [24]:
def updatecenters(dataset,k) :
  centers=[]
  center1=dataset[0] # taking the first center randomly
  centers.append(center1)
  for i in range(k-1) : # this loop is used for calculating the k-1 centers, based on the farthest center approach 
    maxdist=0
    maxpoint=None
    for p in dataset : 
      mindist=np.inf
      for c in centers :
        distance=np.linalg.norm(p-c)
        if distance < mindist :
          mindist=distance 
      if mindist > maxdist :
        maxdist=mindist
        maxpoint=p
    centers.append(maxpoint)
    labels = []
    for p in dataset: # in this loop, we assign each point in the dataset to the respective nearest center forming clusters
        mindist = np.inf
        mincenter_idx = None
        for idx, c in enumerate(centers):
            dist = np.linalg.norm(p - c)
            if dist < mindist:
                mindist = dist
                mincenter_idx = idx
        labels.append(mincenter_idx)   
  return centers,labels


## this below kcenter function is used for calculating the objective value, which is not used in this question

def kcenter(dataset,centers,assign) :
  maxdist=0
  for p,c in zip(dataset,assign) :
    dist=np.linalg.norm(p-c)
    if dist > maxdist :
      maxdist=dist 
  return maxdist 


centers,labels=updatecenters(test_images,k)


rand_index = rand_score(test_labels, labels)
print("Rand Index:", rand_index)

Rand Index: 0.6266329432943294


**part c) Single linkage agglomeration**

In [47]:
from sklearn.metrics import rand_score


def singlelinkage(k, dataset):
    cluster_assignments = []
    number_datapoints = dataset.shape[0]
    ## Initializing each point as its own cluster
    clusters = [[x] for x in dataset]
    dist_matrix = np.zeros((number_datapoints, number_datapoints))

    # assigning a unique cluster id for each clusters
    cluster_ids = [[i] for i in range(number_datapoints)]

    pbar = tqdm(total=len(clusters)-k, desc="Clustering Progress") #tqdm line

    while len(clusters) > k:
        if len(clusters) == number_datapoints: ## calculating distances between all points pairwisely only once(initially), according to my assumption
            for i in range(number_datapoints):
                for j in range(number_datapoints):
                    dist_matrix[i, j] = np.linalg.norm(dataset[i] - dataset[j])
        id1, id2 = np.unravel_index(np.argmin(dist_matrix), dist_matrix.shape) ## taking out the least points with least distance between them 
        cluster_id1, cluster_id2 = sorted([id1, id2])
        for i in cluster_ids[cluster_id1]:  ## this loop is used for nullifying the distances between the one of the above points, as we merge the above point into a single cluster
            dist_matrix[i, cluster_ids[cluster_id2]] = np.inf 
        clusters[cluster_id1].extend(clusters[cluster_id2])
        clusters.pop(cluster_id2) #removing the second point, as it is already merged above in the code 
        cluster_ids[cluster_id1].extend(cluster_ids[cluster_id2])
        cluster_ids.pop(cluster_id2)

        pbar.update(1)

    pbar.close()
  
    for x in tqdm(dataset, desc="Assigning Clusters"):
    # for x in dataset:
            cluster_id = -1
            # Find the cluster id of the closest cluster
            for clust_index, v in enumerate(cluster_ids):
                # Check if the data point is already in the cluster
                if any(np.linalg.norm(dataset[p] - x) == 0 for p in v):
                    cluster_id = clust_index
                    break
            cluster_assignments.append(cluster_id)
    return cluster_assignments

k=10

labels = singlelinkage(k, test_images)
rand_index = rand_score(test_labels, labels)
print("Rand index:", rand_index)


Assigning Clusters: 100%|██████████| 10000/10000 [00:01<00:00, 7692.71it/s]


Rand index: 0.10185978597859786


In [5]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


**Question-4**

In [40]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import entropy 

edges_data = pd.read_csv('facebook_large\musae_facebook_edges.csv')
edges_data.columns = [0,1]
target_data = pd.read_csv('facebook_large\musae_facebook_target.csv')
num_nodes = target_data.shape[0]
target_data.columns = [0,1,2,3]
adj = [[] for i in range(num_nodes)]
degrees = np.zeros(num_nodes)


subgraphs = []
density=[]
entropies=[]


for u,v in zip(edges_data[0], edges_data[1]): # adding undirected edges to our graph (adjacency matrix)
    adj[u].append(v)
    adj[v].append(u)
    degrees[u] +=1
    degrees[v] +=1
    
actual_deg=degrees

def calculate_density(subgraph, degrees):
    num_nodes = len(subgraph)
    if num_nodes == 0:
        return 0
    num = sum(degrees[node] for node in subgraph)
    num=int(num/2) #edges are being counted twice
    return num / len(subgraph) 

def calculate_entropy(subgraph, target_data):
    labels = target_data[3].iloc[subgraph]
    unique_labels, label_counts = np.unique(labels, return_counts=True)
    probabilities = label_counts / len(subgraph)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy


for _ in tqdm(range(5), desc="running greedy algorithm") : #as we have to run the greedy algorithm 5 times by peeling the densest graph everytime
    max_density = -np.inf
    max_density_subgraph = []
    degrees=actual_deg.copy()
    for k in subgraphs: # this loop is peeling off the densest graphs found in the previous iteration
        for node in k :
            degrees[node]=np.inf
            for x in adj[node] : #decreasing the degree by 1, for all the nodes which are connected to the removed node
                degrees[x]-=1
    for iter in range(num_nodes): 
        subgraph = [node for node, degree in enumerate(degrees) if degree != np.inf]
        min_degree_node = np.argmin(degrees)
        ##found the node with minimum degree now
       
    
        density_subgraph = calculate_density(subgraph, degrees)
        if density_subgraph > max_density:
            max_density = density_subgraph
            max_density_subgraph = subgraph
            # print("sriman")
        degrees[min_degree_node] = np.inf ## this below code is for nullifying the removed minimum degree node
        for x in adj[min_degree_node] :
            degrees[x]-=1


    subgraphs.append(max_density_subgraph)
    density.append(max_density)
    entropy=calculate_entropy(max_density_subgraph,target_data)
    entropies.append(entropy)

for i in range(5):
    print(f"Iteration {i + 1} - Density: {density[i]}, Entropy: {entropies[i]}")

running greedy algorithm: 100%|██████████| 5/5 [14:13<00:00, 170.63s/it]

Iteration 1 - Density: 35.00625, Entropy: 0.7893861082107652
Iteration 2 - Density: 25.483516483516482, Entropy: 0.4294619720471056
Iteration 3 - Density: 20.23076923076923, Entropy: 1.13247400192124
Iteration 4 - Density: 16.980392156862745, Entropy: -0.0
Iteration 5 - Density: 15.466666666666667, Entropy: 0.15374218032876188
